In [3]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/accepted_2007_to_2018Q4.csv', sep=",")

/var/folders/fd/wgp9jk8x1msd6_yg7qnrxj0r0000gn/T/ipykernel_17940/704578018.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/accepted_2007_to_2018Q4.csv', sep=",")


In [11]:
def features_generation(df_as_is):
    df = df_as_is.copy()
    df["default"] = np.where(df.loan_status.isin(['Fully Paid', 'Charged Off']), 1, 0)
    df['income_final'] = np.where(df['application_type']=='Joint App',df['annual_inc_joint'],df['annual_inc'])
    df['DTI_final'] = np.where(df['application_type']=='Joint App',df['dti_joint'],df['dti'])
#     df['verification_status_final'] = np.where(df['application_type']=='Joint App',df['verification_status_joint'],df['verification_status'])
    #Соотношение между DTI основного заемщика и общим
    df['DTI_ratio'] = df['DTI_final']/df['dti']

    df['obligations_final'] = df['DTI_final']*df['income_final']/12#annual->month

    #считаем всякие соотношения
#     df['new_obligations_share'] =df['installment']/df['obligations_final'
    df['obligations2bal'] = df['obligations_final']/df['avg_cur_bal']
#     df['installment2bal'] = df['installment']/df['avg_cur_bal']

    #смотрим, с какой точностью указан доход
    df['annual_inc_f'] = np.where(df['income_final'] % 1000 ==0,0,1)

    #восстанавливаем даты и считаем глубину КИ
    df['earliest_cr_line'] = pd.to_datetime('1-'+df.earliest_cr_line, format='%d-%b-%Y')
    df['issue_d'] = pd.to_datetime('1-'+df.issue_d, format='%d-%b-%Y')
    df['CH_duration'] = round((df['issue_d']-df['earliest_cr_line'])/np.timedelta64(1,'M'),0)
    df['CH2Emp'] = round((df['issue_d']-df['earliest_cr_line'])/np.timedelta64(1,'Y'),
                              0)/(np.where(df['emp_length']=='< 1 year',0,df['emp_length'].str[0:1]).astype('float'))
    #делаем таргет бинарным
    df['default'] = np.where(df['default']==True,1,0)

    #zip_code
    df['zip_code_2sign'] = df['zip_code'].str[0:2]
    df['zip_code_1sign'] = df['zip_code'].str[0:1]

    #pub_rec
    df['pub_rec_total'] = df['pub_rec_bankruptcies']+df['pub_rec']

    #desc&emp_title
    df['desc_f'] = np.where(df['desc'].notnull(),1,0)
    df['emp_title_f'] = np.where(df['emp_title'].notnull(),1,0)
    df['title_f'] = np.where(df['title'].notnull(),1,0)

    #inq2CH
    df['inq2CH'] = df['inq_last_6mths']/df['CH_duration']

    #purpose and disbursement_method
    df['purpose_disbursement_method'] = df['disbursement_method']+'_'+df['purpose']
    return(df)

In [12]:
head_df = df[:100]
head_df = features_generation(head_df)

In [10]:
df.loc[df.loan_status.isin(['Fully Paid', 'Charged Off']), : ]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
5,68426831,NaN,11950.0,11950.0,11950.0,36 months,13.44,405.18,C,C3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260688,89905081,NaN,18000.0,18000.0,18000.0,60 months,9.49,377.95,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2260690,88948836,NaN,29400.0,29400.0,29400.0,60 months,13.99,683.94,C,C3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2260691,89996426,NaN,32000.0,32000.0,32000.0,60 months,14.49,752.74,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2260692,90006534,NaN,16000.0,16000.0,16000.0,60 months,12.79,362.34,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
head_df

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,CH_duration,CH2Emp,zip_code_2sign,zip_code_1sign,pub_rec_total,desc_f,emp_title_f,title_f,inq2CH,purpose_disbursement_method
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,148.0,12.000000,19,1,0.0,0,1,1,0.006757,Cash_debt_consolidation
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,192.0,16.000000,57,5,0.0,0,1,1,0.020833,Cash_small_business
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,184.0,15.000000,60,6,0.0,0,1,0,0.000000,Cash_home_improvement
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,87.0,7.000000,07,0,0.0,0,1,1,0.000000,Cash_debt_consolidation
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,210.0,6.000000,17,1,0.0,0,1,1,0.014286,Cash_major_purchase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,68537564,NaN,8000.0,8000.0,8000.0,36 months,10.78,261.08,B,B4,...,215.0,inf,48,4,0.0,0,1,1,0.000000,Cash_debt_consolidation
96,68446784,NaN,12000.0,12000.0,12000.0,36 months,10.78,391.62,B,B4,...,91.0,2.666667,07,0,0.0,0,1,1,0.000000,Cash_credit_card
97,68376533,NaN,18000.0,18000.0,18000.0,36 months,7.49,559.83,A,A4,...,177.0,3.000000,03,0,0.0,0,1,1,0.005650,Cash_debt_consolidation
98,68476714,NaN,3600.0,3600.0,3600.0,36 months,11.48,118.68,B,B5,...,260.0,inf,22,2,2.0,0,1,1,0.007692,Cash_debt_consolidation
